In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import operator

import peakutils

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import operator
from collections import Counter

import scipy.stats
from scipy.optimize import curve_fit
from scipy.misc import factorial

import pickle
from sklearn.feature_extraction.text import CountVectorizer

from statsmodels.discrete.discrete_model import NegativeBinomial, Poisson, Logit

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

In [ ]:
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/word-journey-data.pql"
data_grouped_by_year_label = pd.read_pickle(filename)

In [ ]:
discipline = "Biochemistry & Molecular Biology"
input_ = data_grouped_by_year_label[data_grouped_by_year_label.Label == discipline]
data_grouped_by_year_label = None

In [ ]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(input_.title_words)

vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [ ]:
vectorized_wordarray.shape

In [ ]:
input_["number_of_pubs"] = input_["title_without_stopwords"].progress_apply(lambda x: len(x))
input_["word_count"] = input_["title_words"].progress_apply(lambda x: len(x.split()))

In [ ]:
count_matrix = vectorized_words.toarray()

uniques = []
counts = []
yearly_word_frequency_variations = []
avg_variations_for_freq = []

for i in range(count_matrix.shape[0]):
    unique_, counts_tmp = np.unique(count_matrix[i:i+1,:], return_counts=True)    
    counts_ = list(map(np.log, counts_tmp))
    uniques.append(unique_)
    counts.append(counts_)
    
for index in tqdm(range(vectorized_wordarray.shape[0] - 1)):
    
    # take two consecutive years
    tmp_ = vectorized_wordarray[index:index+2]
    
    #unique counts for the first year
    unique_counts = np.unique(tmp_[0]) 
    
    yearly_avg_variations_for_freq = {}
    word_freq_variations = []
    
    word_freq_variations.append(tmp_[1] - tmp_[0])
    
    yearly_word_frequency_variations.append(word_freq_variations)
    
    for uniq in unique_counts:
        # find the indices where the value == uniq
        indices, = np.where(tmp_[0] == uniq)
        
        # find the values in the next year where those indices are
        # variations of words with that `uniq` count
        transition_values = tmp_[1][indices]
        variations = transition_values - uniq
        
        # mean variation of words with that `uniq` count
        mean_variation = np.mean(variations)
        
        # make a list of mean variations
        yearly_avg_variations_for_freq[uniq] = round(mean_variation, 3)
    
    avg_variations_for_freq.append(yearly_avg_variations_for_freq)    

__Years:__ 2010 - 2015

In [15]:
K = 3
f = plt.figure(figsize=(18,14))


year = 2010
N = 110
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 2011
N = 111
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2012
N = 112
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2013
N = 113
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2014
N = 114
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 2015
N = 115
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 2.110659
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                 3241
Model:               NegativeBinomial   Df Residuals:                     3240
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               6.578e-12
Time:                        15:03:19   Log-Likelihood:                -6840.6
converged:                       True   LL-Null:                       -6840.6
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9939      0.016     62.2

<Figure size 1296x1008 with 0 Axes>

In [16]:
K = 5
f = plt.figure(figsize=(18,14))


year = 2010
N = 110
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 2011
N = 111
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2012
N = 112
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2013
N = 113
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2014
N = 114
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 2015
N = 115
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))


Optimization terminated successfully.
         Current function value: 2.519830
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                 1379
Model:               NegativeBinomial   Df Residuals:                     1378
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               1.354e-12
Time:                        15:04:22   Log-Likelihood:                -3474.8
converged:                       True   LL-Null:                       -3474.8
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5873      0.019     85.4

<Figure size 1296x1008 with 0 Axes>

In [17]:
K = 7
f = plt.figure(figsize=(18,14))


year = 2010
N = 110
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 2011
N = 111
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2012
N = 112
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2013
N = 113
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2014
N = 114
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 2015
N = 115
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 2.700828
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                  754
Model:               NegativeBinomial   Df Residuals:                      753
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               1.081e-11
Time:                        15:04:43   Log-Likelihood:                -2036.4
converged:                       True   LL-Null:                       -2036.4
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9373      0.020     96.0

<Figure size 1296x1008 with 0 Axes>

In [18]:
K = 10
f = plt.figure(figsize=(18,14))


year = 2010
N = 110
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 2011
N = 111
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2012
N = 112
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2013
N = 113
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2014
N = 114
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 2015
N = 115
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 2.978262
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                  412
Model:               NegativeBinomial   Df Residuals:                      411
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               2.396e-11
Time:                        15:05:25   Log-Likelihood:                -1227.0
converged:                       True   LL-Null:                       -1227.0
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3230      0.024     95.7

<Figure size 1296x1008 with 0 Axes>

In [19]:
K = 15
f = plt.figure(figsize=(18,14))


year = 2010
N = 110
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 2011
N = 111
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2012
N = 112
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2013
N = 113
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 2014
N = 114
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 2015
N = 115
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 3.223274
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                  254
Model:               NegativeBinomial   Df Residuals:                      253
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:              -4.382e-11
Time:                        15:05:45   Log-Likelihood:                -818.71
converged:                       True   LL-Null:                       -818.71
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7291      0.026    105.4

<Figure size 1296x1008 with 0 Axes>

In [20]:
K = 10
f = plt.figure(figsize=(18,14))


year = 1975
N = 75
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 1976
N = 76
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1977
N = 77
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1978
N = 78
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1979
N = 79
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 1980
N = 80
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 2.907373
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                  179
Model:               NegativeBinomial   Df Residuals:                      178
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               2.055e-11
Time:                        15:09:26   Log-Likelihood:                -520.42
converged:                       True   LL-Null:                       -520.42
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2083      0.038     57.4

<Figure size 1296x1008 with 0 Axes>

In [21]:
K = 10
f = plt.figure(figsize=(18,14))


year = 1955
N = 55
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
from collections import Counter
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]  
# ax = f.add_subplot(331)
# ax.scatter(X, Y)
# ax.set_xlabel("Variation")
# ax.set_ylabel("Probability")

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

#print(nbModel.loglike(negBinomialRes.params))

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax.scatter(X, Y_poiss, alpha=0.5)
# ax.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



year = 1956
N = 56
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax2 = f.add_subplot(332)
# ax2.scatter(X, Y)
# ax2.set_xlabel("Variation")
# ax2.set_ylabel("Probability")
# ax2.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())

# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax2.scatter(X, Y_poiss, alpha=0.5)
# ax2.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1957
N = 57
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax3 = f.add_subplot(333)
# ax3.scatter(X, Y)
# ax3.set_xlabel("Variation")
# ax3.set_ylabel("Probability")
# ax3.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax3.scatter(X, Y_poiss, alpha=0.5)
# ax3.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1958
N = 58
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax4 = f.add_subplot(334)
# ax4.scatter(X, Y)
# ax4.set_xlabel("Variation")
# ax4.set_ylabel("Probability")
# ax4.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax4.scatter(X, Y_poiss, alpha=0.5)
# ax4.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))




year = 1959
N = 59
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax5 = f.add_subplot(335)
# ax5.scatter(X, Y)
# ax5.set_xlabel("Variation")
# ax5.set_ylabel("Probability")
# ax5.set_title(str(year) + " - Word Variations in " + discipline)


nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax5.scatter(X, Y_poiss, alpha=0.5)
# ax5.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))





year = 1960
N = 60
freq_current = list(vectorized_wordarray[N:N+1,:][0])
freq_next = list(vectorized_wordarray[N+1:N+2,:][0])
indices = [i for i, x in enumerate(freq_current) if x == K]
change_in_values = [freq_next[i] for i in indices]
variations = [j - K for j in change_in_values]
sum_ = sum(dict(Counter(variations)).values())
X = [x + K for x,y in dict(Counter(variations)).items()]
Y = [float(y)/sum_ for x,y in dict(Counter(variations)).items()]
# ax6 = f.add_subplot(336)
# ax6.scatter(X, Y)
# ax6.set_xlabel("Variation")
# ax6.set_ylabel("Probability")
# ax6.set_title(str(year) + " - Word Variations in " + discipline)

nbModel = NegativeBinomial(change_in_values,np.ones_like(change_in_values))
negBinomialRes = nbModel.fit()
print(negBinomialRes.summary())


# lambda_ = np.exp(poissonRes.params[0])
# Y_poiss = list(map(lambda x: poisson(x, lambda_), X))
# ax6.scatter(X, Y_poiss, alpha=0.5)
# ax6.set_title(str(year) + ": " + str(round(poissonModel.loglike(poissonRes.params),2)))



Optimization terminated successfully.
         Current function value: 3.149927
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
                     NegativeBinomial Regression Results                      
Dep. Variable:                      y   No. Observations:                   39
Model:               NegativeBinomial   Df Residuals:                       38
Method:                           MLE   Df Model:                            0
Date:                Thu, 13 Jun 2019   Pseudo R-squ.:               5.580e-11
Time:                        15:14:19   Log-Likelihood:                -122.85
converged:                       True   LL-Null:                       -122.85
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2553      0.104     21.6

<Figure size 1296x1008 with 0 Axes>